### import
- 기준일 다음날 다운받되, 파일명의 날짜는 전일(기준일)로 한다!
- 계좌별진행상황 : 기준일 다음날 아침에 다운받기, 틀린 계좌수정하기, 통합하기, 전산 올리기, 그 결과를 엑셀로 저장하기(기준일까지의 진행상황 반영하기)
  - t+1에 한번만 받으면 됨
  - 현재상황만 실시간이고, 나머지는 전주까지의 기록
  - 따라서 기준일이 금요일인 경우 다음주 월요일(t+1)에 다운 받으면 기준데이터에 입금내역 반영하지 않아도 됨.(전산 opb는 자산정리 통합파일에서 수정)
  - 기준일이 목요일인 경우 금요일(t+1)에 받은 계좌별진행상황에 월~목요일 상환내역을 반영해야함
- 상환내역 : 결과를 엑셀로 저장하기(기준일까지의 진행상황 반영하기)
  - 기준일이 금,토,일요일인 경우, 상환내역을 고려하지 않아도 된다.
  - 기준일이 금,토,일요일이 아닌 경우, 기준일이 속한 주의 월요일 ~ t일 상환내역을 모두 읽어야
  - 여러회차를 한꺼번에 상환할 수 있으니, 마지막 회차, 전체입금원금,이자,기타비용을 계산해야.

In [1]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")

# dtype 정의 (사이트 파일은 읽지 말자. 할 게 너무 많음)
# 나우리 타입 반영
nauri_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
# # 신복 사이트 타입 반영
# nauri_dtype = {'채무자키' : str, '계좌키' : str, '보증인키':str, '신용회복키' : str, '심의차수' : str, '조정전이율' : str, '조정후이율' : str, '총상환기간' : str, '유예기간' : str, '원금균등상환기간' : str, '원리균등상환기간' : str, '이자상환기간' : str, '납입회차' : str, '연체기간' : str, '원금균등시작회차' : str, '원금균등종료회차' : str, '원리균등시작회차' : str, '원리균등종료회차' : str, '이자상환시작회차' : str, '이자상환종료회차' : str, '감면율' : str, '주채무자주민번호' : str, '시작회차1' : str, '종료회차1' : str, '채무조정1적용이율1' : str, '채무조정2적용이율1' : str, '시작회차2' : str, '종료회차2' : str, '채무조정1적용이율2' : str, '채무조정2적용이율2' : str, '시작회차3' : str, '종료회차3' : str, '채무조정1적용이율3' : str, '채무조정2적용이율3' : str, '시작회차4' : str, '종료회차4' : str, '채무조정1적용이율4' : str, '채무조정2적용이율4' : str, '시작회차5' : str, '종료회차5' : str, '채무조정1적용이율5' : str, '채무조정2적용이율5' : str, '시작회차6' : str, '종료회차6' : str, '채무조정1적용이율6' : str, '채무조정2적용이율6' : str, '시작회차7' : str, '종료회차7' : str, '채무조정1적용이율7' : str, '채무조정2적용이율7' : str, '시작회차8' : str, '종료회차8' : str, '채무조정1적용이율8' : str, '채무조정2적용이율8' : str, '시작회차9' : str, '종료회차9' : str, '채무조정1적용이율9' : str, '채무조정2적용이율9' : str, '시작회차10' : str, '종료회차10' : str, '채무조정1적용이율10' : str, '채무조정2적용이율10' : str, '재조정횟수' : str, '수정조정횟수' : str, '일시납감면율' : str, '거치기간' : str, '체증전구간의시작회차' : str, '체증전구간의종료회차' : str, '체증전구간의기준기간' : str, '변제금수취계좌' : str, '담보권실행유예기간':str
# }
deposit_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '담당자키':str, '일시납감면율':str, '계좌번호':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}

### 파일 읽기

In [2]:
# 계좌별 진행상황 파일 읽기
path_base = r"D:\3.자산\신용회복\신용회복 전체데이터\계좌별 진행상황"
########################################################
basedate = "240131"
company = "대성"  # 통합 or 대성, 통합=솔림+신탁88
########################################################
sufix = "_업로드결과"

# 읽을 파일명
filename_read = basedate + "_계좌별 진행상황 조회_" + company + sufix
fullpath_read = join(path_base, filename_read+".xlsx")

# 계좌별 진행상황 조회 파일 읽기
raw_ori = pd.read_excel(fullpath_read, dtype=nauri_dtype).fillna("")

# 계좌 (메모) --------------
if company == "통합" : 
    dir_account = join(r"D:\3.자산\전산 dataset", "솔림", basedate)
else : 
    dir_account = join(r"D:\3.자산\전산 dataset", "대성", basedate)
file_list = os.listdir(dir_account)
account_filename = [file for file in file_list if file.startswith("계좌조회새창")][0]
account = pd.read_excel(join(dir_account, account_filename), dtype=account_dtype)[["계좌키","채무자키","메모"]].fillna("")

In [3]:
# 재시작 위해 복사본 만들기
raw = raw_ori.copy()

# 불필요한 열 삭제
raw.drop(columns=["채무자명","채무상태", "현재채권구분","등록파일","수정파일", "삭제여부","등록일자","등록유저","수정일자","수정유저","확인결과","저장결과"], axis=1, inplace=True)

# 새로운 열을 추가 : insert 여러번 쓰니 성능 경고나와서
진행구분 = pd.Series([None] * len(raw), name='진행구분')
반영OPB = pd.Series([None] * len(raw), name='반영OPB')
상환구분 = pd.Series([None] * len(raw), name='상환구분')

position = 4
raw = pd.concat([raw.iloc[:, :position], 진행구분, 반영OPB, 상환구분, raw.iloc[:, position:]], axis=1)

# 전산미등록건 처리 #####################################################
if company == "통합" : 
    미등록주민번호 = ['771027-2221231'] #강민경
    cond = raw['신청인주민번호'].isin(미등록주민번호)
    raw.loc[cond, '채무자키'] = '전산미등록건'
    raw.loc[cond, '계좌키'] = '전산미등록건'
else :  # 대성
    미등록주민번호 = ['741008-1037838'] #강승욱
    cond = raw['신청인주민번호'].isin(미등록주민번호)
    raw.loc[cond, '채무자키'] = '전산미등록건'
    raw.loc[cond, '계좌키'] = '전산미등록건'
########################################################################

### 진행구분, 반영OPB, 상환구분 작성

In [4]:
# 진행구분열 작성-------------------------------
x = raw.신청인진행상태
y = raw.계좌진행상태내용
z = raw.계좌번호

conds = [
    (x == '정상이행') & (y == '채무조정포함'),
    (y == "채무조정제외"),
    (x.str.contains(r"완제|상환") | y.str.contains(r"완제|상환") | z.str.contains(r"완제|상환")),
    (z == "채권신고전"),
    (x.str.contains(r"심사반송|중도실효|합의서체결포기")),
]
values = ["확정", "채무조정제외","완제","채권신고전","실효"]
default_value = "미확정"

raw["진행구분"] = np.select(conds, values, default_value)

# 반영OPB - 확정미확정 건-------------------------------
x = raw.진행구분
# 확정시 우선순위
a = raw.상환후잔액
b = raw.원금균등채무액
c= raw.조정후원금
d = raw.조정전원금

# 미확정-조정전원금0 은 이자채권으로 그냥 0이 맞다.
conds = [(x == "미확정"), (x =="확정")&(a!=0), b!=0, c!=0, d!=0]

# 조건에 따라 값을 설정
values = [d, a, b, c, d]

# 기본값 설정
default_value = np.nan

# 반영OPB 열 설정
raw["반영OPB"] = np.select(conds, values, default_value)


# 상환구분 작성 - 확정미확정 건-------------------------------
x = raw.상환방식

conds = [x.str.contains("원금균등"), x.str.contains("원리금균등")] #체증식(원금균등)이란게 있네..

# 조건에 따라 값을 설정
values = ["개인","프리"]

# 기본값 설정
default_value = "" # 미확정

# 반영OPB 열 설정
raw["상환구분"] = np.select(conds, values, default_value)

### 시트작성 및 체크 : 최종,정상,제외

In [5]:
# 최종, 확정미확정, 일부계좌제외 시트
최종 = raw.query('진행구분 in ["확정","미확정","실효","완제"]').copy()
확정미확정 = raw.query('진행구분 in ["확정", "미확정"]').copy()
일부계좌제외 = raw.query('진행구분 == "채무조정제외" and 채무자키 != ""') # 계좌번호 변경한 건들 있어서 채무자키 없을 수 있음, 20240117 기준 솔림 8건
# 일부계좌제외는 자산정리 채무상태에서 신복으로 처리하여 오류탐지 제외 ~~

# 전산미등록건 일부계좌제외
최종.drop(최종[최종.채무자키=='전산미등록건'].index, inplace=True)
확정미확정.drop(확정미확정[확정미확정.채무자키=='전산미등록건'].index, inplace=True)
일부계좌제외.drop(일부계좌제외[일부계좌제외.채무자키=='전산미등록건'].index, inplace=True)

# 포함과 제외가 공존하는 제외건만 : 제외자 주민번호가 확정미확정에도 있으면
일부계좌제외 = 일부계좌제외.merge(확정미확정["신청인주민번호"], how="inner", on=["신청인주민번호"])

# 순번열 넣기
if "순번" not in raw.columns : 
    raw.insert(0, "순번", range(1,len(raw)+1))
    최종.insert(0, "순번", range(1,len(최종)+1))
    확정미확정.insert(0, "순번", range(1,len(확정미확정)+1))
    일부계좌제외.insert(0, "순번", range(1,len(일부계좌제외)+1))

In [6]:
# 체크
# 반영OPB
반영OPB오류 = raw[(raw.반영OPB == np.nan)|(raw.반영OPB == 0)]
cols = ["채무자키", "계좌키", "진행구분", "상환구분","반영OPB", "조정전원금"]
if len(반영OPB오류) > 0 : 
    print('반영OPB 0원 ------')
    print('원금,법비용 0원계좌라도 상환예정액 조금이라도 나옴. 미확정건이면 전산opb를 따르니 0원 되겠지') ######################
    display(반영OPB오류[cols])
else : print("반영OPB 오류 없음")

# 확정미확정에 신용회복키 중복여부
신용회복키중복 = 확정미확정[확정미확정.duplicated('신용회복키', keep=False)]
if len(신용회복키중복) > 0 : 
    print('신용회복키 중복건 ------ 신용회복키 빈값 없애려면 업로드 후에 재다운로드해야')
    display(신용회복키중복)
else : print("확정미확정에 신용회복키 중복 없음")

# 채무자키, 계좌키, 신용회복키 매칭 오류
최종키오류 = 최종.query('채무자키=="" or 계좌키=="" or 신용회복키==""')
확정미확정키오류 = 확정미확정.query('채무자키=="" or 계좌키=="" or 신용회복키==""')
print(f"전체 : {len(raw)}, 최종 : {len(최종)}, 확정미확정 : {len(확정미확정)}")
print(f"최종에서 키 없는 행 수 : {len(최종키오류)}, 확정미확정에서 키없는 행 수 : {len(확정미확정키오류)}")
if len(확정미확정키오류)>0 : 
    print('확정미확정 채무자키, 계좌키, 신복키 중 하나라도 없는 경우 ------')
    display(확정미확정키오류)
else : print("확정미확정 건에 키매칭 실패 없음")

# 확정 건 중 상환구분 없는 값
print("확정 건 중 상환구분 없는 값 :", len(확정미확정.query('상환방식=="확정" and 상환구분==""')))

# 일부계좌제외에 보증채무있는지
print("일부계좌제외에 보증채무수 :", len(일부계좌제외.query('채무구분=="보증채무"'))) # 있으면 드랍할지 말지 그때 결정

반영OPB 오류 없음
확정미확정에 신용회복키 중복 없음
전체 : 984, 최종 : 954, 확정미확정 : 663
최종에서 키 없는 행 수 : 6, 확정미확정에서 키없는 행 수 : 0
확정미확정 건에 키매칭 실패 없음
확정 건 중 상환구분 없는 값 : 0
일부계좌제외에 보증채무수 : 0


### 상환내역 - 금요일 기준이면 pass
- 기준일이 금욜이 아닌경우 월~기준일까지 입금내역을 반영해야 함
- 프리의 경우 정확한 상환후잔액은 산정이 불가해서 기준일은 금요일로만 할 것을 권장

In [7]:
############# 원리금균등 : 조정후합계 - 총납입원금 + 발생이자, 여기서 이자를 정확히 구하기 어려우므로 가급적 금요일을 자산정리 할 것을 권장 ##################
# 회차 때문에 전산 입금내역이 아니라 신용회복 변제금 상환내역 파일이 필요함
# 상환내역 파일읽기
# 폴더경로
if company == "대성" : 
    deposit_base = r"D:\3.자산\신용회복\대성\신용회복\입금"
else : 
    deposit_base = r"D:\3.자산\신용회복\솔림\1.신용회복\0.입금관련,담당자변경\2024년"

# 파일명 리스트 ############################################################################################
# 월요일 ~ 기준일까지의 입금 업로드결과 파일(하나의 파일로 합쳐도 되고, 개별로 올려도 되고)
deposit_list = ["대성 신용회복 입금_240129~31_업로드결과"] 
###########################################################################################################

# 입금 파일 읽기
deposit_ori = pd.DataFrame(None)
for f in deposit_list : 
    temp = pd.read_excel(join(deposit_base, f+".xlsx"), dtype=deposit_dtype).fillna("")
    deposit_ori = pd.concat([deposit_ori, temp], axis=0)

# 컬럼명 변경
deposit_ori.rename(columns={'입금자':'신청인명','주민번호':'신청인주민번호', 
'입금자구분':'채무구분','입금원금':'납입원금','입금미수이자':'납입이자','입금연체이자':'납입기타채무','입금합계':'납입합계'}, inplace=True)

In [8]:
# 복사본
deposit = deposit_ori.copy()

In [9]:
# 입금업로드결과 신용회복키 수정 : 한 채무자키에 여러 신용회복이 등록된 경우 첫번째 신용회복키로 통일되어버리는 문제

# 채무구분 수정 : 입금했는데 채무구분이 빈칸일 리는 없으니 채무자 > 주채무, 보증인 > 보증채무
# 채구구분값 통일
deposit["채무구분"] = np.where(deposit.채무구분=="채무자", "주채무", "보증채무")
# 새로운 단일키 조합
최종["대체신용회복키"] = 최종.계좌번호 + 최종.신청인주민번호 + 최종.채무구분
deposit["대체신용회복키"] = deposit.계좌번호 + deposit.신청인주민번호 + deposit.채무구분

# 병합
deposit = pd.merge(deposit, 최종[["신용회복키","대체신용회복키"]], how='left', on='대체신용회복키')

# 대체신용회복키 삭제(분납키는 그냥 두자..)
del deposit["대체신용회복키"]

##### 상환내역 반영 : 기준일 금요일이면 pass

In [10]:
# 실효,완제건 입금 리스트에서 제거
temp1 = len(deposit)
deposit = deposit.drop(deposit.query('~신용회복키.isin(@확정미확정.신용회복키)').index).reset_index(drop=True)
temp2 = len(deposit)
print(f'실효완제건수 : {temp1-temp2}')
# 입금을 납입회차 역순정렬 후 신용회복키로 그룹화
deposit.sort_values(['신용회복키','납입회차'], ascending=[True,False], inplace=True)

실효완제건수 : 1


In [11]:
# 그룹 순회하면서 기준데이터(확정미확정)에 다음을 반영
# 수정할 값 : 납입회차(첫번재 즉 가장 큰 수로 대체), 총납입금액의 합계를 합산, 총납입원금의 합계를 합산, 총납입이자의 합계를 합산, 총납입기타채무의 합계를 합산, 기존 상환후잔액 - 총납입원금의 합계
수정전금액들 = pd.Series([확정미확정.총납입금액.sum(), 확정미확정.총납입원금.sum(), 확정미확정.총납입이자.sum(), 확정미확정.총납입기타채무.sum(), 확정미확정.상환후잔액.sum()])
현재신용회복키 = ""
for i, v in tqdm(deposit.iterrows(), total=len(deposit)) :
    # 기준데이터 수정할 행
    ind = 확정미확정[확정미확정.신용회복키 == v.신용회복키].index
    if 현재신용회복키 != v.신용회복키 : # 신용회복키가 다른 경우만 납입회차 수정
        현재신용회복키 = v.신용회복키
        확정미확정.loc[ind, '납입회차'] = v.납입회차
        # 입금수정표시
        
    
    # 금액은 합산이므로 신용회복키 중복값에 상관없이 계산 적용하면된다
    확정미확정.loc[ind, '총납입금액'] += v.납입합계
    확정미확정.loc[ind, '총납입원금'] += v.납입원금
    확정미확정.loc[ind, '총납입이자'] += v.납입이자
    확정미확정.loc[ind, '총납입기타채무'] += v.납입기타채무
    확정미확정.loc[ind, '상환후잔액'] -= v.납입원금

수정후금액들 = pd.Series([확정미확정.총납입금액.sum(), 확정미확정.총납입원금.sum(), 확정미확정.총납입이자.sum(), 확정미확정.총납입기타채무.sum(), 확정미확정.상환후잔액.sum()])
수정한금액들 = pd.Series([deposit.납입합계.sum(), deposit.납입원금.sum(), deposit.납입이자.sum(), deposit.납입기타채무.sum(), -deposit.납입원금.sum()])

# 총액이 잘 맞는지 확인
print((수정후금액들 == 수정전금액들 + 수정한금액들).unique())

  0%|          | 0/163 [00:00<?, ?it/s]

100%|██████████| 163/163 [00:00<00:00, 489.94it/s]

[ True]


### 통합신고건 처리하기

In [12]:
통합신고계좌 = account.query("메모.str.contains('신용회복 통합신고')")
print("통합신고계좌수 :",len(통합신고계좌))
print("추가전 확정미확정 건수 :", len(확정미확정))

cnt = 확정미확정.순번.iloc[-1]

for i, v in 통합신고계좌.iterrows() : 
    피통합계좌 = re.search(r"[0-9]{9,9}", v.메모)[0]
    temp = 확정미확정.query('계좌키 == @피통합계좌').copy()
    if len(temp) == 1 :
        cnt += 1
        temp['순번'] = cnt
        temp['채무자키'] = v.채무자키
        temp['계좌키'] = v.계좌키
        temp[['신용회복키','계좌진행상태내용']] = '통합신고 ' + v.계좌키
        temp['반영OPB'] = 0
        
        # 행추가
        확정미확정 = pd.concat([확정미확정, temp])
    elif len(temp) == 0 :
        print("통합신고 취소, 원상회복 분할적용 확인 : ", v.계좌키, v.채무자키)
    
    else : 
        print("중복계좌키 발견 : ", v.계좌키, v.채무자키)
        
print("추가후 확정미확정 건수 :", len(확정미확정))

전체요약 = 확정미확정.groupby('진행구분').agg({'신용회복키':'count', '반영OPB':'sum'})
전체요약.loc["합계", :] = 전체요약.sum()
display(전체요약.applymap('{:,.0f}'.format))

통합신고계좌수 : 0
추가전 확정미확정 건수 : 663
추가후 확정미확정 건수 : 663


,신용회복키,반영OPB
진행구분,,
미확정,4,"12,595,322"
확정,659,"758,184,577"
합계,663,"770,779,899"


### 파일 저장

In [13]:
# 저장
# 기준데이터 저장 경로
filename_write = basedate+"_신용회복_기준데이터_"+ ("솔림" if company=="통합" else "대성")
fullpath_result = join(os.path.dirname(path_base),"기준데이터", filename_write +".xlsx")


# 엑셀 파일로 저장
if not os.path.exists(fullpath_result):
    param = {'mode' : 'w'}
else : 
    param = {'mode' : 'a', 'if_sheet_exists' : 'replace'}

with pd.ExcelWriter(fullpath_result, engine='openpyxl', **param) as writer :
    raw.to_excel(writer, sheet_name='원본', index=False)
    최종.to_excel(writer, sheet_name='최종', index=False)
    확정미확정.to_excel(writer, sheet_name='확정,미확정', index=False)
    일부계좌제외.to_excel(writer, sheet_name='일부계좌제외', index=False)